# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


In [1]:
import xlwings as xw
import numpy as np
import pandas as pd

In [2]:
#!pip install xlwings==0.24.9 --user

In [3]:
xw.__version__

'0.24.9'

## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [4]:
wb = xw.Book('себестоимостьА_в1.xlsx')

In [5]:
sheet = wb.sheets['Рецептура']

In [6]:
consumption = sheet.range('G7:O10').options(np.array).value
unit_price = sheet.range('G14:O14').options(np.array).value

In [7]:
consumption

array([[0.2  , 0.4  , 0.02 , 0.02 , 0.01 , 0.005, 0.1  ,   nan,   nan],
       [0.07 , 0.25 , 0.01 , 0.06 , 0.015,   nan, 0.06 , 0.05 ,   nan],
       [0.12 , 0.22 , 0.005,   nan, 0.009, 0.005,   nan, 0.05 , 0.03 ],
       [0.12 , 0.23 , 0.01 ,   nan, 0.015,   nan,   nan,   nan, 0.05 ]])

In [8]:
unit_price

array([  7.,  10., 184.,  19.,   7., 290., 105.,  83., 260.])

In [9]:
costs = np.nan_to_num(consumption *unit_price).sum(axis=1)
costs

array([21.48 , 16.525, 17.423, 18.085])

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

In [10]:
sheet.range('T7:T10').options(transpose = True).value = costs

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

In [11]:
sheet.range('T6').value ='Себестоимость'
sheet.range('T4:T6').api.Merge()

4. Выполнить 3 с помощью "протягиваемых" формул.

In [12]:
from xlwings.constants import AutoFillType

In [13]:
sheet.range('U7').formula = '=SUMPRODUCT(G7:O7, $G$14:$O$14)'
sheet.range('U7').api.AutoFill(sheet.range('U7:U10').api, AutoFillType.xlFillDefault)

True

## Лабораторная работа 7

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [14]:
recipes = pd.read_csv('recipes_sample_with_tags_ingredients.csv', usecols=['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients'], parse_dates=['submitted'], index_col=1)
recipes.head()

,name,minutes,submitted,description,n_ingredients
id,,,,,
44123,george s at the cove black bean soup,90,2002-10-25,an original recipe created by chef scott meska...,18
67664,healthy for them yogurt popsicles,10,2003-07-26,my children and their friends ask for my homem...,3
38798,i can t believe it s spinach,30,2002-08-29,"these were so go, it surprised even me.",8
35173,italian gut busters,45,2002-07-27,my sister-in-law made these for us at a family...,9
84797,love is in the air beef fondue sauces,25,2004-02-23,i think a fondue is a very romantic casual din...,12


In [15]:
reviews = pd.read_csv('reviews_sample.csv', parse_dates=['date'])
reviews.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
reviews.set_index('id', inplace=True)
reviews.head()

,user_id,recipe_id,date,rating,review
id,,,,,
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [16]:
wb = xw.Book('recipes.xlsx')

In [17]:
#rec_sheet = wb.sheets.add(name="Рецепты") #создание
#rev_sheet = wb.sheets.add(name="Отзывы") 

In [18]:
rec_sheet = wb.sheets['Рецепты']
rev_sheet = wb.sheets['Отзывы']

In [19]:
rec_sheet.range((1, 1)).value = recipes.sample(frac = 0.05)
rev_sheet.range((1, 1)).value = reviews.sample(frac = 0.05)

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [20]:
rec_len = rec_sheet.range((1, 1)).end('down').row
rev_len = rev_sheet.range((1, 1)).end('down').row

In [21]:
last_col = rec_sheet.range((1, 1)).end('right').column
headers = rec_sheet.range((1, 1)).expand('right').value

In [22]:
def find_col(headers, name):
    for i in range(len(headers)):
        if headers[i] == name:
            return i + 1

In [23]:
rec_sheet.range((1, last_col + 1)).value = 'seconds_assign'
headers = rec_sheet.range((1, 1)).expand('right').value
minutes_ind = find_col(headers, 'minutes')
minutes = rec_sheet.range((2, minutes_ind), (rec_len, minutes_ind)).options(np.array).value
rec_sheet.range((2, last_col + 1), (rec_len, last_col + 1)).options(transpose = True).value = minutes*60

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [24]:
last_col = rec_sheet.range((1, 1)).end('right').column
rec_sheet.range((1, last_col + 1)).value = 'seconds_formula'
headers = rec_sheet.range((1, 1)).expand('right').value
rec_sheet.range((2, last_col + 1)).formula = '=C2*60'
rec_sheet.range((2, last_col + 1)).api.AutoFill(rec_sheet.range((2, last_col + 1), (rec_len, last_col + 1)).api, AutoFillType.xlFillDefault)

True

5. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [25]:
last_col = rec_sheet.range((1, 1)).end('right').column
rec_sheet.range((1, last_col + 1)).value = 'n_reviews'
headers = rec_sheet.range((1, 1)).expand('right').value

In [26]:
rec_sheet.range((2, last_col + 1)).formula = '=COUNTIF(Отзывы!$C$2:$C$6336,A2)'
rec_sheet.range((2, last_col + 1)).api.AutoFill(rec_sheet.range((2, last_col + 1), (rec_len, last_col + 1)).api, AutoFillType.xlFillDefault)

True

6. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [27]:
from xlwings.constants import HAlign

In [28]:
rec_sheet.range('G1').expand('right').api.Font.Bold = True
rec_sheet.range('G1').expand('right').api.HorizontalAlignment = HAlign.xlHAlignCenter

7. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [29]:
minutes_ind = find_col(headers, 'minutes')
df = rec_sheet.range((1, minutes_ind), (rec_len, minutes_ind)).options(pd.DataFrame).value
df.reset_index(level=0, inplace=True)
df['color'] = "(255, 255, 0)"
df.loc[df['minutes'] < 5,'color'] = "(0, 255, 0)"
df.loc[df['minutes'] > 5,'color'] = "(255, 0, 0)"
df.head()

,minutes,color
0,70.0,"(255, 0, 0)"
1,290.0,"(255, 0, 0)"
2,6.0,"(255, 0, 0)"
3,25.0,"(255, 0, 0)"
4,330.0,"(255, 0, 0)"


In [30]:
for i in range(2, rec_len + 1):
    rec_sheet.range((i, minutes_ind)).color = eval(df['color'][i - 2])

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [31]:
def validate():
    rev_headers = rev_sheet.range((1, 1)).expand('right').value
    rating_ind = find_col(rev_headers, 'rating')
    rec_id_ind = find_col(rev_headers, 'recipe_id')
    df = rev_sheet.range((1, rec_id_ind), (rev_len, rating_ind)).options(pd.DataFrame).value
    df.reset_index(level=0, inplace=True)
    df.drop('date', axis=1, inplace=True)
    rec_ids = rev_sheet.range((2, find_col(headers, 'id')), (rec_len, find_col(headers, 'id'))).options(np.array).value
    df['status'] = (df['rating'] >= 0) & (df['rating'] <= 5) & (df['recipe_id'].apply(lambda x: x in rec_ids))
    for i in range(2, rev_len + 1):
        if (not df['status'][i - 2]):
            rev_sheet.range((i, 1)).expand("right").color = (255, 139, 139)

In [32]:
validate()